In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ! pip install langchain
# ! pip install langchain_openai

In [70]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableLambda, RunnableBranch

In [79]:
api_key= 'your_api_key'

In [12]:
model = ChatOpenAI(api_key=api_key)

# 1. Runnable Sequence

In [13]:
prompt = PromptTemplate(
    template='List out top 5 series from the genre :{genre} ',
    input_variables=['genre']
)

In [14]:
str_parser = StrOutputParser()

In [44]:
seq_runnable1 = RunnableSequence(prompt, model, str_parser)

# result = seq_runnable1.invoke({'genre': 'anime'})

# print(type(result))

In [57]:
prompt2 = PromptTemplate(
    template="List a male and a female character from each of the series: {series}",
    input_variables=['series']
)

In [51]:
seq_runnable2 = RunnableSequence(seq_runnable1, prompt2, model, str_parser)

result = seq_runnable2.invoke({'genre':'anime'})

print(result)

1. Levi Ackerman
2. Izuku Midoriya (Deku)
3. Naruto Uzumaki
4. Monkey D. Luffy
5. Edward Elric


# 2. Runnable Parallel

In [52]:
prompt3 = PromptTemplate(
    template = "List top 5 latest released series on crunchyroll of the genre: {genre}",
    input_variables=['genre']
)

In [53]:
parallel_chain = RunnableParallel({
    'top_5': RunnableSequence(prompt, model, str_parser),
    'crunchy_roll_release': RunnableSequence(prompt3, model, str_parser)
})

result = parallel_chain.invoke({'genre': 'anime'})

print(result)
print("\n")
print(f"top_5: {result['top_5']}")
print(f"crunchy_roll_release: {result['crunchy_roll_release']}")

{'top_5': '1. Attack on Titan\n2. Naruto\n3. My Hero Academia\n4. One Piece\n5. Dragon Ball Z', 'crunchy_roll_release': '1. Attack on Titan Final Season Part 1\n2. Jujutsu Kaisen \n3. The Rising of the Shield Hero Season 2\n4. Tokyo Revengers \n5. My Hero Academia Season 5'}


top_5: 1. Attack on Titan
2. Naruto
3. My Hero Academia
4. One Piece
5. Dragon Ball Z
crunchy_roll_release: 1. Attack on Titan Final Season Part 1
2. Jujutsu Kaisen 
3. The Rising of the Shield Hero Season 2
4. Tokyo Revengers 
5. My Hero Academia Season 5


# 3. Runnable Passthrough

In [62]:

runnable5 = RunnableParallel({
    'series_list': RunnablePassthrough(),
    'characters': RunnableSequence(prompt2, model, str_parser)
})

runnable6 = RunnableSequence(seq_runnable1, runnable5)

result = runnable6.invoke({'genre': 'anime'})

print(f" series_list : {result_5['series_list']}")
print("\n")

print(f"characters: {result_5['characters']}")

 series_list : 1. "Attack on Titan"
2. "Naruto"
3. "One Piece"
4. "My Hero Academia"
5. "Death Note"


characters: 1. "Attack on Titan":
- Male character: Eren Yeager
- Female character: Mikasa Ackerman

2. "Naruto":
- Male character: Naruto Uzumaki
- Female character: Sakura Haruno

3. "One Piece":
- Male character: Monkey D. Luffy
- Female character: Nami

4. "My Hero Academia":
- Male character: Izuku Midoriya (Deku)
- Female character: Ochaco Uraraka

5. "Death Note":
- Male character: Light Yagami
- Female character: Misa Amane


# 4. Runnable Lambda

In [68]:
def word_count(text):
    print(text.split())
    
    return len(text.split())

In [69]:
runnable_word_counter = RunnableLambda(word_count)

runnable7 = RunnableSequence(seq_runnable1, RunnableParallel({
    'output': RunnablePassthrough(),
    'word_count': runnable_word_counter
    }))

result = runnable7.invoke({'genre': 'anime'})

print(result)

['1.', 'Fullmetal', 'Alchemist:', 'Brotherhood', '2.', 'Attack', 'on', 'Titan', '3.', 'Naruto', '4.', 'Death', 'Note', '5.', 'My', 'Hero', 'Academia']
{'output': '1. Fullmetal Alchemist: Brotherhood\n2. Attack on Titan\n3. Naruto\n4. Death Note\n5. My Hero Academia', 'word_count': 17}


# 5. Runnable Branch

In [77]:
def output_n(text):
    return 'Naruto in top 5'


def output_d(text):
    return 'Demon Slayer in top 5'


def output_t(text):
    return 'Attack on Titans in top 5'

def output_f():
    return 'None in top 5'


runnable8 = RunnableBranch(
    (lambda x: 'Naruto' in x.split(), RunnableLambda(output_n)),
    (lambda x: 'Demon' in x.split(),  RunnableLambda(output_d)),
    (lambda x: 'Titan' in x.split(),  RunnableLambda(output_t)),
    (RunnableLambda(output_f))
)

runnable9 = RunnableParallel({
    'branching': runnable8,
    'series': RunnablePassthrough()
    })

#runnable10 = RunnableSequence(seq_runnable1, runnable9)

runnable10 = seq_runnable1 | runnable9 # LCEL

result = runnable10.invoke({"genre": 'anime'})

print(result, "\n")

print(result['series'], "\n")


print(result['branching'], "\n")

{'branching': 'Naruto in top 5', 'series': '1. Attack on Titan\n2. My Hero Academia\n3. Naruto\n4. One Piece\n5. Death Note'} 

1. Attack on Titan
2. My Hero Academia
3. Naruto
4. One Piece
5. Death Note 

Naruto in top 5 



# LCEL - Langchain Expression Language

In [78]:
# representing 
#   RunnableSequence(seq_runnable1, runnable9) 
#           as 
#   seq_runnable1 | runnable9  
#
# is an expression used in langchain